# Chapter 5: Support Vector Machines

Support Vector Machines (SVM) merupakan salah satu algoritma Machine Learning yang **sangat kuat dan serbaguna**. SVM dapat digunakan untuk classification, regression, hingga outlier detection.

Keunggulan utama SVM terletak pada kemampuannya untuk membangun **decision boundary dengan margin terbesar**, sehingga model memiliki kemampuan generalisasi yang baik.

Pada chapter ini, kita akan mempelajari:
- konsep *large margin classification*,
- perbedaan hard margin dan soft margin,
- SVM linear dan non-linear,
- kernel trick,
- SVM untuk regresi,
- serta gambaran matematis cara kerja SVM.

Pemahaman SVM sangat penting karena konsep margin dan kernel menjadi dasar bagi banyak metode Machine Learning lanjutan.

## 1. Linear SVM Classification

Ide fundamental di balik SVM adalah **large margin classification**, yaitu mencari decision boundary yang tidak hanya memisahkan kelas dengan benar, tetapi juga berada sejauh mungkin dari instance terdekat dari setiap kelas.

Decision boundary yang memiliki margin besar cenderung lebih stabil dan lebih mampu melakukan generalisasi terhadap data baru.

Instance data yang berada tepat di tepi margin disebut **support vectors**. Menariknya, hanya support vectors inilah yang menentukan posisi decision boundary.

### 1.1 Sensitivity to Feature Scales

SVM sangat **sensitif terhadap skala fitur**. Jika satu fitur memiliki skala yang jauh lebih besar dibandingkan fitur lain, maka decision boundary dapat menjadi bias.

Oleh karena itu, **feature scaling** (misalnya menggunakan StandardScaler) merupakan langkah yang hampir selalu diperlukan sebelum melatih model SVM.

## 2. Soft Margin Classification

Pada kondisi ideal, semua data dapat dipisahkan secara sempurna oleh decision boundary (hard margin). Namun, pada data nyata:
- data sering tidak sepenuhnya linearly separable,
- terdapat noise dan outlier.

Untuk mengatasi hal ini, SVM menggunakan pendekatan **soft margin**, yaitu mengizinkan sebagian pelanggaran margin dengan tujuan mendapatkan model yang lebih robust.

Trade-off antara margin yang besar dan jumlah pelanggaran margin dikontrol oleh hyperparameter **C**.

Jika nilai **C kecil**, model akan lebih toleran terhadap pelanggaran margin dan cenderung memiliki margin yang lebih besar.

Sebaliknya, jika nilai **C besar**, model akan berusaha mengklasifikasikan semua data dengan benar, namun berisiko overfitting.

## 3. Linear SVM Classification with Scikit-Learn

Pada bagian ini, kita akan melatih sebuah Linear SVM untuk melakukan binary classification menggunakan dataset Iris.

Tujuan model adalah mendeteksi apakah suatu bunga termasuk spesies **Iris virginica** atau bukan.

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

In [ ]:
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

svm_clf.fit(X, y)

Pipeline di atas memastikan bahwa data distandarisasi terlebih dahulu sebelum masuk ke model Linear SVM.

Penggunaan `loss="hinge"` sesuai dengan formulasi klasik SVM yang memaksimalkan margin.

### 3.1 Making Predictions

Setelah model dilatih, kita dapat menggunakannya untuk melakukan prediksi pada data baru.

In [ ]:
svm_clf.predict([[5.5, 1.7]])

Hasil prediksi berupa nilai kelas, bukan probabilitas. Berbeda dengan Logistic Regression, SVM **tidak secara langsung menghasilkan probabilitas kelas**.

## 4. Nonlinear SVM Classification

Tidak semua dataset dapat dipisahkan menggunakan decision boundary linear. Pada kasus seperti ini, **Linear SVM** tidak mampu menghasilkan model yang baik.

Untuk mengatasi masalah tersebut, SVM dapat diperluas menjadi **Nonlinear SVM**, yaitu dengan cara memproyeksikan data ke ruang berdimensi lebih tinggi sehingga data menjadi *linearly separable* di ruang baru tersebut.

### 4.1 Polynomial Features

Salah satu cara paling sederhana untuk menangani data non-linear adalah dengan menambahkan **fitur polinomial**.

Dengan menambahkan kombinasi non-linear dari fitur asli, model linear dapat mempelajari decision boundary yang non-linear di ruang input asli.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

Dataset *moons* sering digunakan sebagai contoh klasik data non-linear. Dua kelas data membentuk pola setengah lingkaran yang tidak dapat dipisahkan oleh garis lurus.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)

Dengan menambahkan fitur polinomial, model Linear SVM kini mampu memisahkan data non-linear.

Namun, pendekatan ini memiliki kelemahan: jumlah fitur meningkat sangat cepat seiring bertambahnya derajat polinomial.

## 5. The Kernel Trick (Pengantar)

Untuk menghindari eksplisit menambahkan fitur polinomial dalam jumlah besar, SVM menggunakan teknik yang disebut **kernel trick**.

Kernel trick memungkinkan kita menghitung hubungan di ruang berdimensi tinggi **tanpa benar-benar memetakan data ke ruang tersebut**.

Intinya, kernel trick membuat SVM tetap efisien secara komputasi meskipun decision boundary yang dihasilkan bersifat non-linear di ruang input asli.

Pada bagian selanjutnya, kita akan langsung menggunakan kernel SVM yang tersedia di Scikit-Learn, tanpa perlu menambahkan fitur polinomial secara manual.

## 6. Polynomial Kernel

Alih-alih menambahkan fitur polinomial secara eksplisit, SVM menyediakan **Polynomial Kernel** yang menerapkan *kernel trick*.

Dengan polynomial kernel, SVM dapat mempelajari decision boundary non-linear secara efisien, bahkan pada ruang fitur berdimensi sangat tinggi.

Polynomial kernel memiliki beberapa hyperparameter penting:
- **degree**: menentukan derajat polinomial (kompleksitas decision boundary)
- **coef0**: mengontrol pengaruh fitur tingkat rendah vs tingkat tinggi
- **C**: mengontrol regularization (trade-off margin vs kesalahan)

Pemilihan hyperparameter ini sangat memengaruhi performa dan kompleksitas model.

In [ ]:
from sklearn.svm import SVC

poly_kernel_svm_clf = SVC(kernel="poly", degree=3, coef0=1, C=5)
poly_kernel_svm_clf.fit(X, y)

Model di atas menggunakan polynomial kernel dengan derajat 3. Dengan kombinasi parameter ini, SVM mampu mempelajari pola non-linear pada dataset *moons*.

### 6.1 Pengaruh Parameter `degree`

Parameter `degree` menentukan tingkat kompleksitas decision boundary.

- Degree rendah menghasilkan boundary yang lebih sederhana
- Degree tinggi memungkinkan boundary yang lebih kompleks, namun meningkatkan risiko overfitting

Pemilihan nilai degree yang tepat sangat bergantung pada struktur data.

### 6.2 Pengaruh Parameter `coef0`

Parameter `coef0` mengontrol kontribusi fitur-fitur tingkat rendah dalam polynomial kernel.

Nilai `coef0` yang besar meningkatkan pengaruh fitur tingkat rendah, sedangkan nilai kecil menekankan fitur tingkat tinggi.

### 6.3 Pengaruh Parameter `C`

Parameter `C` mengatur seberapa ketat model berusaha mengklasifikasikan seluruh data dengan benar.

- Nilai `C` kecil menghasilkan margin yang lebih lebar dan model yang lebih sederhana
- Nilai `C` besar menghasilkan margin yang lebih sempit dan model yang lebih kompleks

Trade-off ini sangat penting untuk menghindari overfitting maupun underfitting.

## 7. Gaussian Radial Basis Function (RBF) Kernel

Salah satu kernel yang paling populer dan paling sering digunakan dalam SVM adalah **Gaussian Radial Basis Function (RBF) kernel**.

RBF kernel memungkinkan SVM membentuk decision boundary yang sangat fleksibel, bahkan pada dataset dengan struktur yang kompleks dan tidak teratur.

Secara intuitif, RBF kernel mengukur **kedekatan (similarity)** antara dua instance. Semakin dekat dua titik data, semakin besar nilai kernel-nya.

### 7.1 Intuisi RBF Kernel

RBF kernel dapat dibayangkan sebagai cara untuk menempatkan *Gaussian bell* di sekitar setiap instance data.

Instance yang berada dekat satu sama lain akan memiliki pengaruh yang kuat, sementara instance yang jauh pengaruhnya akan mengecil.

Pendekatan ini membuat SVM dengan RBF kernel mampu mempelajari decision boundary yang sangat non-linear.

### 7.2 Hyperparameter `gamma`

Parameter **`gamma`** mengontrol seberapa jauh pengaruh sebuah instance data menjangkau instance lainnya.

- **Gamma kecil** → pengaruh instance sangat luas → decision boundary lebih halus
- **Gamma besar** → pengaruh instance sangat sempit → decision boundary lebih kompleks

Nilai gamma yang terlalu besar dapat menyebabkan overfitting, sedangkan nilai yang terlalu kecil dapat menyebabkan underfitting.

### 7.3 RBF Kernel SVM with Scikit-Learn

Scikit-Learn menyediakan implementasi RBF kernel melalui class `SVC`. Dengan memilih kernel `rbf`, kita dapat langsung menggunakan RBF kernel tanpa perlu transformasi fitur manual.

In [ ]:
rbf_kernel_svm_clf = SVC(kernel="rbf", gamma=0.5, C=1)
rbf_kernel_svm_clf.fit(X, y)

Model di atas menggunakan RBF kernel dengan nilai `gamma=0.5` dan `C=1`. Kombinasi kedua parameter ini menentukan tingkat fleksibilitas decision boundary.

Pada praktiknya, `gamma` dan `C` sering ditentukan menggunakan teknik *hyperparameter tuning* seperti grid search.

### 7.4 Kapan Menggunakan RBF Kernel?

RBF kernel merupakan pilihan default yang baik ketika:
- hubungan antar fitur bersifat non-linear
- jumlah fitur tidak terlalu besar
- tidak ada pengetahuan awal tentang bentuk decision boundary

Karena fleksibilitasnya, RBF kernel sering memberikan performa yang sangat baik pada berbagai jenis dataset.

## 8. Support Vector Machine for Regression (SVR)

Selain digunakan untuk classification, Support Vector Machines juga dapat digunakan untuk **regression**, yang dikenal sebagai **Support Vector Regression (SVR)**.

Berbeda dengan regresi klasik yang meminimalkan error prediksi secara langsung, SVR berusaha mencari fungsi regresi yang berada dalam batas toleransi tertentu terhadap data training.

### 8.1 Intuisi Support Vector Regression

Pada SVR, model berusaha memuat sebanyak mungkin instance data **di dalam sebuah margin toleransi** yang ditentukan oleh parameter $\varepsilon$.

- Error yang berada di dalam margin $\varepsilon$ **tidak dikenakan penalti**
- Error di luar margin akan dikenakan penalti

Dengan pendekatan ini, SVR fokus pada **kesederhanaan model** dan **robustness terhadap noise**.

### 8.2 Linear SVR

Linear SVR mencoba menemukan fungsi linear yang memprediksi target dengan deviasi maksimum sebesar $\varepsilon$.

Parameter **C** tetap berperan sebagai pengontrol trade-off antara lebar margin dan penalti terhadap error besar.

In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y.ravel())

Model di atas mencoba memprediksi nilai target dengan mengabaikan error kecil di dalam margin toleransi.

Pendekatan ini membuat SVR relatif tahan terhadap noise ringan pada data.

### 8.3 Nonlinear SVR

Seperti pada classification, SVR juga dapat diperluas menjadi **nonlinear regression** dengan menggunakan kernel, seperti polynomial kernel atau RBF kernel.

Hal ini memungkinkan SVR memodelkan hubungan non-linear antara fitur dan target.

In [ ]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y.ravel())

Dengan menggunakan polynomial kernel, SVR mampu menangkap pola non-linear pada data regresi.

Pemilihan kernel dan hyperparameter sangat memengaruhi performa model.

## Closing Summary (Chapter 5)

Chapter 5 membahas Support Vector Machines sebagai algoritma yang kuat dan fleksibel untuk berbagai jenis tugas Machine Learning.

Konsep utama yang dipelajari meliputi:
- large margin classification,
- soft margin dan regularization,
- kernel trick untuk menangani non-linearitas,
- serta penerapan SVM pada classification dan regression.

Pemahaman SVM memberikan wawasan penting tentang bagaimana model yang kompleks dapat dikontrol agar tetap memiliki kemampuan generalisasi yang baik.

Konsep margin, kernel, dan regularization yang diperkenalkan pada chapter ini juga menjadi fondasi penting untuk memahami model Machine Learning lanjutan pada chapter berikutnya.